In [58]:
import pandas as pd
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
import torch 
from visual import find_opt_thres, get_metric_invariant, load_results
from matplotlib import pyplot as plt
from visual import load_csv
# evaluator = Evaluator(name='ogbl-collab')
evaluator_hit = Evaluator(name='ogbl-collab')
evaluator_mrr = Evaluator(name='ogbl-citation2')

ImportError: cannot import name 'find_opt_thres' from 'visual' (/Users/ruiruiwang/KIT/project/TAPE/analysis/visual.py)

In [ ]:
# Example usage
FILE_PATH = '/hkfs/work/workspace/scratch/cc7738-benchmark_tag/educational_demo/'
file_path = FILE_PATH + 'err_ncnc_llama/ncnc-cora_AUC_0.9669_MRR_0.5275.csv'
P1, P2, pos_index, neg_index = load_results(file_path)
best_thres_llama, best_acc_llama, pos_pred_llama, neg_pred_llama = find_opt_thres(P1, P2)


plt.figure(figsize=(12, 8))
# Plot distributions of probabilities
plt.hist(P2, bins=100, alpha=0.5, color='blue', label='Neg Class')
plt.hist(P1, bins=100, alpha=0.5, color='red', label='Pos Class')
best_thres, best_acc, pos_pred, neg_pred = find_opt_thres(P1, P2)

plt.axvline(best_thres, color='green', linestyle='--', label=f'Optimal Threshold = {best_thres:.2f}')
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.legend()
plt.title('Probability Distributions with Optimal Threshold')
plt.savefig('optimal_threshold.png')
print(f'best_accuracy: {best_acc}, best_threshold: {best_thres}')

k_list  = [0.1, 0.2, 0.3, 0.5, 1]
pos_index = torch.tensor(pos_index)
neg_index = torch.tensor(neg_index)
P1 = torch.tensor(P1)
P2 = torch.tensor(P2)
mrr_pos2neg, mrr_neg2pos, result_auc_test, pos_edge_index_err, pos_rank_err, neg_edge_index_err, neg_rank_err = get_metric_invariant(P1, pos_index, P2, neg_index, k_list)

print(mrr_pos2neg)
print(result_auc_test)
print(mrr_neg2pos)


In [ ]:
import os
import sys

# Assuming your target directory is one level up from the current working directory
notebook_dir = os.getcwd()  
target_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

sys.path.insert(0, target_dir)
from core.data_utils.load import load_data_lp
from core.graphgps.utility.utils import init_cfg_test

cfg = init_cfg_test()

In [ ]:
splits, text, data = load_data_lp[cfg.data.name](cfg.data)

In [ ]:
splits

In [ ]:
data

In [ ]:
print(f'index predicted to be neg when pos : {pos_edge_index_err.shape}')
print(f'index predicted to be pos when neg : {neg_edge_index_err.shape}')

## What Pos is predicted as Neg? Type II.

In [ ]:
for row in pos_edge_index_err:
    print(f'source: {text[row[0]]}, \n target: {text[row[1]]}')

In [ ]:
from IPython.display import display, Markdown

for row in pos_edge_index_err:
    src = text[row[0]]
    tgt = text[row[1]]
    display(Markdown(f"**Source:** {src}  \n**Target:** {tgt}"))

## Analysis for Error I
https://24timezones.com/difference/frankfurt_ger/new_delhi?dt=2024-07-31T830pm 
1. Such citation indeed exists, however, it appears in a historical section.
| ![Example Image](intro.png) |![Example Image](ref.png) |

## Analysis for Error II 
1. textual features are quite noisy. 

**Target:**

**Title:** Cholinergic Suppression of Transmission May Allow Combined Associative Memory Function and Self-Organization in the Neocortex

**Abstract:** Selective suppression of transmission at feedback synapses during learning is proposed as a mechanism for combining associative feedback with self-organization of feedforward synapses. Experimental data demonstrates cholinergic suppression of synaptic transmission in layer I (feedback synapses), and a lack of suppression in layer IV (feed-forward synapses). A network with this feature uses local rules to learn mappings which are not linearly separable. During learning, sensory stimuli and desired responses are simultaneously presented as input. Feedforward connections form self-organized representations of input, while suppressed feedback connections learn the transpose of feedforward connectivity. During recall, suppression is removed, sensory input activates the self-organized representation, and activity generates the learned response.

### No idea what the hell is this? Title is the Figure description and abstract is followed by the reference?

**Source:**

**Title:** Figure 8: Time Complexity of Unit Parallelism Measured on MANNA Theoretical Prediction #nodes N Time

**Abstract:** Our experience showed us that flexibility in expressing a parallel algorithm for simulating neural networks is desirable even if it is not possible then to obtain the most efficient solution for any single training algorithm. We believe that the advantages of a clear and easy-to-understand program predominate the disadvantages of approaches allowing only for a specific machine or neural network algorithm. We currently investigate if other neural network models are worthwhile being parallelized, and how the resulting parallel algorithms can be composed of a few common basic building blocks and the logarithmic tree as an efficient communication structure.

**References:**
1. D. Ackley, G. Hinton, T. Sejnowski: *A Learning Algorithm for Boltzmann Machines*, Cognitive Science 9, pp. 147-169, 1985
2. B. M. Forrest et al.: *Implementing Neural Network Models on Parallel Computers*, The Computer Journal, vol. 30, no. 5, 1987
3. W. Giloi: *Latency Hiding in Message Passing Architectures*, International Parallel Processing Symposium, April 1994, Cancun, Mexico, IEEE Computer Society Press
4. T. Nordström, B. Svensson: *Using And Designing Massively Parallel Computers for Artificial Neural Networks*, Journal Of Parallel And Distributed Computing, vol. 14, pp. 260-285, 1992
5. A. Kramer, A. Vincentelli: *Efficient Parallel Learning Algorithms for Neural Networks*, Advances in Neural Information Processing Systems I, D. Touretzky (ed.), pp. 40-48, 1989
6. T. Kohonen: *Self-Organization and Associative Memory*, Springer-Verlag, Berlin, 1988
7. D. A. Pomerleau, G. L. Gusciora, D. L. Touretzky, H. T. Kung: *Neural Network Simulation at Warp Speed: How We Got 17 Million Connections Per Second*, IEEE Intern. Conf. Neural Networks, July 1988
8. A. Rübel: *Dynamic Selection of Training Patterns for Neural Networks: A New Method to Control the Generalization*, Technical Report 92-39, Technical University of Berlin, 1993
9. D. E. Rumelhart, D. E. Hinton, R. J. Williams: *Learning Internal Representations by Error Propagation*, Rumelhart & McClelland (eds.), Parallel Distributed Processing: Explorations in the Microstructure of Cognition, vol. I, pp. 318-362, Bradford Books/MIT Press, Cambridge, MA, 1986
10. W. Schiffmann, M. Joost, R. Werner: *Comparison of Optimized Backpropagation Algorithms*, Proc. of the European Symposium on Artificial Neural Networks, ESANN '93, Brussels, pp. 97-104, 1993
11. J. Schmidhuber: *Accelerated Learning in BackPropagation Nets*, Connectionism in Perspective, Elsevier Science Publishers B.V. (North-Holland), pp 439-445, 1989
12. M. Taylor, P. Lisboa (eds.): *Techniques and Applications of Neural Networks*, Ellis Horwood, 1993
13. M. Witbrock, M. Zagha: *An Implementation of Backpropagation Learning on GF11, a Large SIMD Parallel Computer*, Parallel Computing, vol. 14, pp. 329-346, 1990
14. X. Zhang, M. McKenna, J. P. Mesirov, D. L. Waltz: *The Backpropagation Algorithm on Grid and Hypercube Architectures*, Parallel Computing, vol. 14, pp. 317-327, 1990

In [ ]:
from IPython.display import display, Markdown

for row in neg_edge_index_err:
    src = text[row[0]]
    tgt = text[row[1]]
    display(Markdown(f"**Source:** {src}  \n**Target:** {tgt}"))

In [ ]:
def adamic_adar(adj, node1, node2):
    """计算Adamic-Adar指数"""
    neighbors1 = adj[node1].nonzero(as_tuple=False).flatten()
    neighbors2 = adj[node2].nonzero(as_tuple=False).flatten()
    common_neighbors = torch.tensor(list(set(neighbors1.tolist()).intersection(set(neighbors2.tolist()))))
    index = adj.sum(dim=0).squeeze()
    aa_score = sum(1.0 / torch.log1p(index[neighbor].float()) for neighbor in common_neighbors)
    return aa_score

def resource_allocation(adj, node1, node2):
    """计算资源分配指数"""
    neighbors1 = adj[node1].nonzero(as_tuple=False).flatten()
    neighbors2 = adj[node2].nonzero(as_tuple=False).flatten()
    common_neighbors = torch.tensor(list(set(neighbors1.tolist()).intersection(set(neighbors2.tolist()))))
    index = adj.sum(dim=0).squeeze()
    ra_score = sum(1.0 / index[neighbor].float() for neighbor in common_neighbors)
    return ra_score

def pagerank(adj, node, alpha=0.85, max_iter=100):
    """计算PageRank得分"""
    n = adj.size(0)
    ranks = torch.ones(n, dtype=torch.float32) / n
    for _ in range(max_iter):
        new_ranks = (1 - alpha) / n + alpha * adj @ ranks
        if torch.allclose(new_ranks, ranks, atol=1e-6):
            break
        ranks = new_ranks
    return ranks[node]

def katz(adj, node1, node2, alpha=0.005, beta=0.005, max_iter=1000):
    """计算Katz得分"""
    n = adj.size(0)
    I = torch.eye(n)
    katz_matrix = alpha * I + beta * adj
    katz_scores = torch.linalg.pinv(katz_matrix) @ torch.ones(n, dtype=torch.float32)

    return katz_scores[node1] + katz_scores[node2]

def shortest_path_length(adj, node1, node2):
    """计算最短路径长度"""
    n = adj.size(0)
    dist = torch.full((n, n), float('inf'))
    dist[torch.arange(n), torch.arange(n)] = 0
    for i in range(adj.size(0)):
        for j in range(adj.size(0)):
            if adj[i, j] != 0:
                dist[i, j] = 1
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i, j] > dist[i, k] + dist[k, j]:
                    dist[i, j] = dist[i, k] + dist[k, j]
    return dist[node1, node2]

In [ ]:
from torch_geometric import utils
from data_utils.lcc import use_lcc

splits, text, data = load_data_lp[cfg.data.name](cfg.data)
# new_data, lcc_index, G = use_lcc(data)
G = utils.to_dense_adj(data.edge_index).squeeze()
G

In [ ]:
pos_edge_index_err.shape

In [ ]:
for row in pos_edge_index_err:
    src = row[0]
    tgt = row[1]
    try:
        print("Adamic-Adar:", adamic_adar(G, src, tgt))
        print("Resource Allocation:", resource_allocation(G, src, tgt))
        print("PageRank:", pagerank(G, src))
        print("Katz:", katz(G, src, tgt))
        print("Shortest Path Length:", shortest_path_length(G, src, tgt))
    except:
        print("Error")

In [ ]:
for row in neg_edge_index_err:
    src = row[0]
    tgt = row[1]
    print("Adamic-Adar:", adamic_adar(G, src, tgt))
    print("Resource Allocation:", resource_allocation(G, src, tgt))
    print("PageRank:", pagerank(G, src))
    print("PageRank:", pagerank(G, tgt))
    print("Katz:", katz(G, src, tgt))

In [ ]:
count = splits['test']['pos_edge_label_index'].T.size(0)
AA = 0
RA = 0
Katz = 0
for row in splits['test']['pos_edge_label_index'].T:
    src = row[0]
    tgt = row[1]
    AA += adamic_adar(G, src, tgt)
    RA += resource_allocation(G, src, tgt)
    Katz += katz(G, src, tgt)
    
print("mAA:", AA/count)
print("mRA:", RA/count)
print("mKatz:", Katz/count)